# Critique extraction

In [14]:
from utils import load_openai_key

load_openai_key()

In [15]:
from langchain_core.prompts import ChatPromptTemplate

prompt_msg = ChatPromptTemplate.from_template("""
    Extract all the issues mentioned in the review and return them as a list. Only extract concrete issues, not general sentiments. I.e. "The battery life is too short" is an issue, but "I don't like the design" is not.
                                              Or "not made to last long" is not an issue, but "(because) screws are loose" is an issue.

    Review:
    {input}
""")

In [16]:
from langchain_core.pydantic_v1 import BaseModel, Field


class Review(BaseModel):
    """Collection of issues mentioned in a review"""
    critique_points: list[str] = Field(description="List of issues mentioned in the review")

In [17]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125").with_structured_output(Review)

In [18]:
chain = prompt_msg | llm

In [19]:
from pathlib import Path


reviews = list(Path('ReviewFetcher/raw_reviews').glob('*.txt'))
len(reviews)

30

In [20]:
import json


review = json.loads(reviews[0].read_text().replace("'", '"'))

In [21]:
review

{'title': '3,0 von 5 Sternen\nToll, aber!',
 'text': 'Meine Kinder lieben den Würfel, leider ist er vermutlich nicht dafür gebaut, wirklich lange und ständig verwendet zu werden. Die geklebte Folie löst sich nun ab.',
 'rating': '3,0 von 5 Sternen',
 'date': '25. Mai 2024'}

In [27]:
import locale


def parse_to_date_object(date_str: str) -> datetime:
    locale.setlocale(locale.LC_TIME, 'de_DE')
    date_obj = datetime.strptime(date_str, "%d. %B %Y")
    return date_obj

In [28]:
from dataclasses import dataclass
from datetime import datetime


@dataclass
class ReviewInput:
    title: str
    text: str
    rating: int
    date: datetime
    
    @property
    def prompt_txt(self) -> str:
        return f"Title: {self.title}\nText: {self.text}\nRating: {self.rating}/5\nDate: {self.date.strftime('%Y-%m-%d')}"

    @classmethod
    def from_json(cls, data: dict):
        title = " ".join(review['title'].split('\n')[1:])
        rating = int(review['rating'][0])
        date = parse_to_date_object(review['date'])

        return cls(title, review['text'], rating, date)


In [29]:
review_input = ReviewInput.from_json(review)

In [33]:
print(review_input.prompt_txt)

Title: Toll, aber!
Text: Meine Kinder lieben den Würfel, leider ist er vermutlich nicht dafür gebaut, wirklich lange und ständig verwendet zu werden. Die geklebte Folie löst sich nun ab.
Rating: 3/5
Date: 2024-05-25


In [34]:
def extract_issues_from_review(review: ReviewInput) -> Review:
    return chain.invoke({"input": review.prompt_txt})

extract_issues_from_review(review_input)

Review(critique_points=['Die geklebte Folie löst sich ab'])

In [39]:
reviews[0].stem[7:]

'0'

In [47]:
issues = {}
for review_file in reviews:
    print(f"Processing {review_file.stem}")
    review = json.loads(review_file.read_text().replace('"', '\\"').replace("'", '"'))
    review_input = ReviewInput.from_json(review)
    extracted = extract_issues_from_review(review_input)

    issues[review_file.stem[7:]] = extracted.critique_points

Processing review_0
Processing review_1
Processing review_10
Processing review_11
Processing review_12
Processing review_13
Processing review_14
Processing review_15
Processing review_16
Processing review_17
Processing review_18
Processing review_19
Processing review_2
Processing review_20
Processing review_21
Processing review_22
Processing review_23
Processing review_24
Processing review_25
Processing review_26
Processing review_27
Processing review_28
Processing review_29
Processing review_3
Processing review_4
Processing review_5
Processing review_6
Processing review_7
Processing review_8
Processing review_9


In [48]:
issues

{'0': ['Die geklebte Folie löst sich ab'],
 '1': ['Dauerhaltbarkeit könnte deutlich besser sein'],
 '10': ['nur eine leere Packung erhalten',
  'nichts drin außer das Handbuch und die Verpackung'],
 '11': ['Die Folie auf dem Würfel ist nach zwei Monaten gerissen',
  'Dafür ist er zu teuer'],
 '12': ['viel zu teuer verkauft'],
 '13': ['nach 5 Minuten gelangweilt',
  'nichtmal aufbekommen',
  'mit Gewalt öffnen und kaputt machen',
  'nur aus Pappe',
  'zu teuer'],
 '14': ['Folie löst sich nach einem Monat',
  'Preis zu hoch für überzogenen Kunststoff',
  'Folie komplett abgerissen nach mehreren Wochen',
  'Würfel unbrauchbar'],
 '15': ['Etwas klein',
  'keine gute Haptik',
  'Preis von 25 € scheint mir etwas überteuert',
  'enttäuscht weg gepackt',
  'letztendlich zurück geschickt'],
 '16': ['Folie geht auseinander',
  'Teile halten nur mit Folie zusammen',
  'Preis zu hoch'],
 '17': ['kleiner als erwartet', 'für den Preis ein wenig simpel'],
 '18': ['Sehr klein', 'Sehr euch die Maße gen